In [1]:
import sys
import os
# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [2]:
# Loading Config
from utils.config import load_yaml_config, merge_configs, dict_to_namespace
import numpy as np

default_yaml = "../utils/configs/default.yaml"
problem_yaml = "../utils/configs/problem_3.yaml"
category_yaml = "../utils/configs/experiment_trnc_c.yaml"

custom_config_files = [default_yaml, problem_yaml, category_yaml] # Add more config files if needed
configs = [load_yaml_config(config_file) for config_file in custom_config_files]
yaml = merge_configs(*configs)

## Here you can modify the yaml object to change the configuration
## Example: yaml.seed = 123, yaml.max_num_tasks = 10
config = dict_to_namespace(yaml)

In [3]:
from env.cades_env import CadesEnv
from stable_baselines3.common.env_checker import check_env

# Initialize and check the environment
env = CadesEnv(config)
check_env(env)

In [4]:
config.train = False
config.evaluate = True

In [ ]:
from models.ppo import PPOModel

model_url = "../../experiments/models/p3/trnc_c/early_term_1000"
# model_url = "../mlruns/304206841489822493/8113f929570e45b489b2e7395b9d5873/artifacts/models/epoch_1000"
model = PPOModel.load(model_url, env, config)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
%ls

chain_comm.ipynb  evaluate_heuristic.ipynb  runtime_complexity.ipynb


In [7]:
# redefine model's predict method to use the heuristic
from heuristics.ffd import FirstFitDecreasingHeuristic

heuristic = FirstFitDecreasingHeuristic(env)
model.model.predict = heuristic.predict

In [8]:
model.evaluate_multiple()

{'mean_episode_reward': 107.88055555555556,
 'mean_episode_length': 9.01,
 'mean_inference_time': 0.0009640789031982422,
 'termination_cause': {'success': 100.0,
  'duplicate_pick': 0.0,
  'node_overflow': 0.0,
  'duplicate_critical_pick': 0.0},
 'mean_metrics': {'avg_node_occupancy': 58.463,
  'avg_active_node_occupancy': 82.85050000000001,
  'message_channel_occupancy': 84.5961,
  'empty_nodes': 28.726700000000005}}